In [1]:
using LinearAlgebra # Nioce.
# open file 
mni = open("input.txt","r");
raw = readlines(mni)
scans = []
cur_scan = Matrix{Int}(undef, 0, 3)
for line in raw
    if line == ""
        push!(scans, cur_scan)
        cur_scan = Matrix{Int}(undef, 0, 3)
        continue
    elseif line[1:3] == "---"
        continue
    end
    cur_scan = [cur_scan; reshape([parse(Int, val) for val in split(line,",")],1,3)]
end
  
push!(scans, cur_scan);

In [2]:
function Rx(θ)
    rot = zeros(Int,3,3)
    rot[1,1] = 1
    rot[2,2] = cosd(θ)
    rot[2,3] = sind(θ)
    rot[3,2] = -sind(θ)
    rot[3,3] = cosd(θ)
    return rot
end
function Ry(θ)
    rot = zeros(Int,3,3)
    rot[1,1] = cosd(θ)
    rot[1,3] = -sind(θ)
    rot[2,2] = 1
    rot[3,1] = sind(θ)
    rot[3,3] = cosd(θ)
    return rot
end
function Rz(θ)
    rot = zeros(Int,3,3)
    rot[1,1] = cosd(θ)
    rot[1,2] = sind(θ)
    rot[2,1] = -sind(θ)
    rot[2,2] = cosd(θ)
    rot[3,3] = 1
    return rot
end

Rz (generic function with 1 method)

In [3]:
const Rxc = Rx(90)
const Ryc = Ry(90)
const Rzc = Rz(90)
const neg_x = [-1 0 0; 0 1 0; 0 0 1]
const neg_y = [1 0 0; 0 -1 0; 0 0 1]
const neg_z = [1 0 0; 0 1 0; 0 0 -1];

In [4]:
function translate(scan, t)
    T = t .* ones(Int, size(scan, 1), size(scan, 2))
    t_scan = scan .+ T
    return t_scan
end

function overlap(scan, full_map)
    overlap = Matrix{Int}(undef, 0, 3)
    for s in 1:size(scan, 1)
        scan_pt = transpose(scan[s,1:3])
        for m in 1:size(full_map, 1)
            map_pt = transpose(full_map[m,1:3])
            
            if scan_pt == map_pt
                overlap = [overlap; scan_pt]
            end
        end
    end
    
    return overlap
end
function get_translations(scan, full_map)
    translations = []
    for s in 1:size(scan, 1)
        scan_pt = transpose(scan[s,1:3])
        for m in 1:size(full_map, 1)
            map_pt = transpose(full_map[m,1:3])
            T = map_pt - scan_pt
            push!(translations, T)
        end
    end
    return translations
end
function get_rotations()
    rotations = []
    push!(rotations, I)
    push!(rotations, Rzc)
    push!(rotations, Rzc*Rzc)
    push!(rotations, Rzc*Rzc*Rzc)
    push!(rotations, Ryc)
    push!(rotations, Ryc*Ryc*Ryc)
    
    push!(rotations, Rxc)
    push!(rotations, Rxc*Rzc)
    push!(rotations, Rxc*Rzc*Rzc)
    push!(rotations, Rxc*Rzc*Rzc*Rzc)
    push!(rotations, Rxc*Ryc)
    push!(rotations, Rxc*Ryc*Ryc*Ryc)
    
    push!(rotations, Rxc*Rxc)
    push!(rotations, Rxc*Rxc*Rzc)
    push!(rotations, Rxc*Rxc*Rzc*Rzc)
    push!(rotations, Rxc*Rxc*Rzc*Rzc*Rzc)
    push!(rotations, Rxc*Rxc*Ryc)
    push!(rotations, Rxc*Rxc*Ryc*Ryc*Ryc)
    
    push!(rotations, Rxc*Rxc*Rxc)
    push!(rotations, Rxc*Rxc*Rxc*Rzc)
    push!(rotations, Rxc*Rxc*Rxc*Rzc*Rzc)
    push!(rotations, Rxc*Rxc*Rxc*Rzc*Rzc*Rzc)
    push!(rotations, Rxc*Rxc*Rxc*Ryc)
    push!(rotations, Rxc*Rxc*Rxc*Ryc*Ryc*Ryc)
    
    return rotations
end
function merge_scan(scan, full_map)
    new_map = copy(full_map)
    
    for s in 1:size(scan, 1)
        scan_pt = transpose(scan[s,1:3])
        exists = false
        for m in 1:size(full_map, 1)
            map_pt = transpose(full_map[m,1:3])
            
            if scan_pt == map_pt
                exists = true
                break
            end
        end
        
        if !exists
           new_map = [new_map; scan_pt]
        end
    end
    return new_map
end
function register_scan(scan, full_map)
    full_map = copy(full_map)
    rotation = I
    translation = I
    for R in get_rotations()
        rotated = scan*R
        translations = get_translations(rotated, full_map)
        for T in translations
            #@show T
            rot_trans_scan = translate(rotated, T)
            common_beacons =  overlap(rot_trans_scan, full_map)
            if length(common_beacons) >= 12*3
                full_map = merge_scan(rot_trans_scan, full_map)
                rotation = R
                translation = T
                return full_map, rotation, translation
            end
        end
     end
    return full_map, rotation, translation
end

register_scan (generic function with 1 method)

In [ ]:
ss = copy(scans)
full_map = popfirst!(scans)
registered_scans = []

while length(ss) > 1
    scan = popfirst!(ss)
    new_map, R, T = register_scan(scan, full_map);
    if new_map == full_map
        push!(ss, scan)
    else
        full_map = new_map
        push!(registered_scans, (scan, R, T))
    end
end
length(full_map)/3

In [17]:
all_T = [scan[3] for scan in registered_scans]
mh_dists = []
for T1 in all_T
    for T2 in all_T
        push!(mh_dists, sum(abs.(T1 .- T2)))
    end
end

sort!(mh_dists, rev = true)[1]

9655